In [1]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=12000)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
from tifffile import imsave, imread, imwrite
from keras.models import Input, Model
from datagenerator_2D import data_generator
from model_2D_interconnect import make_generator

1 Physical GPUs, 1 Logical GPUs


In [2]:
model_save_directory =  r"D:\Projects\Denoising-STED\20220913-RPI\tubulin-histone\histon_2step.h5" 
save_image_dr = r"D:\Projects\Denoising-STED\20220913-RPI\photobleaching\results-histon" 

In [11]:
x_test = imread(r'D:\Projects\Denoising-STED\20220913-RPI\photobleaching\low SNR\1frame.tif').astype(np.float64)[:,0,:,:]

In [12]:
# x_test = imread(r'D:\Projects\Denoising-STED\20220913-RPI\tubulin-clathrine-highthroughput\1frame.tif')
# for i in range(len(x_test)):
#     imwrite(r'D:\Projects\Denoising-STED\20220913-RPI\tubulin-clathrine-highthroughput\tubulin\low SNR\image_'+str(i)+'.tif',x_test[i])

In [13]:
for i in range(len(x_test)):
    x_test[i] = x_test[i]/x_test[i].max()
x_test = x_test.reshape(x_test.shape+(1,))

In [14]:
filters =[64,128,256]
num_filters = filters[0]
filters_cab=num_filters/8
num_RG=3
num_RCAB=8

patch_size = 2048

generator_input = Input((patch_size, patch_size,1))
generator = make_generator(generator_input, filters, num_filters,filters_cab,num_RG,num_RCAB,
                           kernel_shape=3,dropout=0.2)
generator.load_weights(model_save_directory)

prediction2 = np.zeros(x_test.shape)

for i in range(len(x_test)):
    _,prediction2[i]= generator(x_test[i:i+1],training=False)
    pred2_test = prediction2[i]
    pred2_test = pred2_test/pred2_test.max()
    pred2_test[pred2_test<0]=0
    pred2_test = pred2_test*(2**16-1)
    pred2_test = pred2_test.astype(np.uint16)
    imwrite(save_image_dr+'\image_'+str(i)+'.tif', pred2_test.squeeze())


In [ ]:
ix = np.random.randint(len(prediction1))
# ix = 0
fig = plt.figure(figsize=(40,40))

plt.subplot(1, 4, 1)
plt.imshow(x_test[ix, :, :, 0] , cmap='magma')
plt.title('Low SNR Input',fontdict={'fontsize':20})
plt_axis = plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(prediction1[ix, :, :, 0] , cmap='magma')
plt.title('Prediction by UNet',fontdict={'fontsize':20})
plt_axis = plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(prediction2[ix, :, :, 0] , cmap='magma')
plt.title('Prediction by RCAN',fontdict={'fontsize':20})
plt_axis = plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(y_test[ix, :, :, 0] , cmap='magma')
plt.title('Ground Truth',fontdict={'fontsize':20})
plt_axis = plt.axis('off')